<a href="https://colab.research.google.com/github/viswakimi/Multiclass-Fish-Image-Classification/blob/main/Multiclass_Fish_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, MobileNet, InceptionV3, EfficientNetB0
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import streamlit as st
from PIL import Image

In [ ]:
# Define Constants
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 20
DATASET_PATH = "dataset/"
MODEL_SAVE_PATH = "best_fish_model.h5"

In [ ]:
# Data Preprocessing & Augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_gen = datagen.flow_from_directory(
    DATASET_PATH, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', subset='training'
)
val_gen = datagen.flow_from_directory(
    DATASET_PATH, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', subset='validation'
)

In [ ]:
# Function to Create CNN Model
def create_cnn_model(input_shape, num_classes):
    model = Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
# Function to Load and Fine-Tune Pretrained Models
def load_pretrained_model(base_model, num_classes):
    base_model = base_model(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
    for layer in base_model.layers:
        layer.trainable = True  # Fine-tune all layers
    x = Flatten()(base_model.output)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
# Training Models
num_classes = len(train_gen.class_indices)
models = {
    "CNN": create_cnn_model((*IMG_SIZE, 3), num_classes),
    "VGG16": load_pretrained_model(VGG16, num_classes),
    "ResNet50": load_pretrained_model(ResNet50, num_classes),
    "MobileNet": load_pretrained_model(MobileNet, num_classes),
    "InceptionV3": load_pretrained_model(InceptionV3, num_classes),
    "EfficientNetB0": load_pretrained_model(EfficientNetB0, num_classes)
}

In [ ]:
history_dict = {}
metrics_dict = {}
best_model_name = ""
best_accuracy = 0.0

for model_name, model in models.items():
    print(f"Training {model_name}...")
    checkpoint = ModelCheckpoint(MODEL_SAVE_PATH, save_best_only=True, monitor='val_accuracy', mode='max')
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    history = model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS, callbacks=[checkpoint, early_stopping])
    history_dict[model_name] = history.history
    model.save(f"{model_name}_fish_model.h5")

    # Evaluate model
    val_preds = np.argmax(model.predict(val_gen), axis=1)
    val_labels = val_gen.classes

    accuracy = accuracy_score(val_labels, val_preds)
    metrics_dict[model_name] = {
        "Accuracy": accuracy,
        "Precision": precision_score(val_labels, val_preds, average='macro'),
        "Recall": recall_score(val_labels, val_preds, average='macro'),
        "F1 Score": f1_score(val_labels, val_preds, average='macro'),
        "Confusion Matrix": confusion_matrix(val_labels, val_preds)
    }

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model_name = model_name

print(f"Best Model: {best_model_name} with Accuracy: {best_accuracy:.4f}")

In [ ]:
# Visualize Training History Separately
for model_name, history in history_dict.items():
    fig, axs = plt.subplots(1, 2, figsize=(12, 4))

    axs[0].plot(history['accuracy'], label='Train Accuracy')
    axs[0].plot(history['val_accuracy'], label='Validation Accuracy')
    axs[0].set_title(f'{model_name} Accuracy')
    axs[0].set_xlabel('Epochs')
    axs[0].set_ylabel('Accuracy')
    axs[0].legend()

    axs[1].plot(history['loss'], label='Train Loss')
    axs[1].plot(history['val_loss'], label='Validation Loss')
    axs[1].set_title(f'{model_name} Loss')
    axs[1].set_xlabel('Epochs')
    axs[1].set_ylabel('Loss')
    axs[1].legend()

    plt.show()
